# Adavanced Kinetic Modeling
## Target analysis $-$ Defining an own kinetic model

In [ ]:
!pip install kimopack
!pip install python-pptx
!git clone https://github.com/erdzeichen/KiMoPack.git

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
try:
    import KiMoPack.plot_func as pf
except:
    print("General installation did not work, import from the same folder as a workaround")
    import plot_func as pf

## 1) Import data

In [ ]:
solvent = 'ACN' #'DCM' or 'H2O'

filename = 'TA_Ru-dppz_400nm_'+str(solvent)                # set name of the file to fit
filepath = os.path.join('KiMoPack','Tutorial_Notebooks','Data', 'Fitting-2')# set path to file to fit
ta=pf.TA(filename=filename+'.SIA',    # title of the measurement file
         path=filepath)               # path to measuremnt file

#Alternative:
# ta=pf.TA('gui')   #and navigate to the corresponding file

## 2) Standard corrections

In [ ]:
ta.Filter_data(value=20)                # remove artificial values
ta.Background(uplimit=-0.5)             # subtract background before time zero
ta.Cor_Chirp(shown_window=[-2.3,1.8])   # choose time-window used in the active plot

## 3) Plot pre-processed data

In this example the pre-processed data is visualized in three plots (as indicated in the titles), 

1. as kinetic traces (x: $\Delta{}t$, y: $\Delta$Absorbance)
2. transient spectra (x: $\lambda_{probe}$, y: $\Delta$Absorbance)
3. 2D-contour plot (x: $\lambda_{probe}$, y: $\Delta{}t$, z: $\Delta$Absorbance). 

Several features can be used to alter the appearance of those plots (see Documentation or type ```ta.Plot?``` in the notebook). 

* The parameters ```rel_time``` and ```rel_wave``` are used to pre-select interesting $\Delta{}t$ and $\lambda_{probe}$ values to show specific kinetic traces (```plotting=1```) or transient spectra (```plotting=2```)  of the dataset. 
* The parameters ```timelimits```, ```bordercut``` and ```intensity_range``` are specified to control the displayed region, by specifying upper and lower limits of delay times, probe wavelengths and TA signal intensities, respectively. 
* The scale of the TA signals can be changed to a logarithmic scaling using \code{log\_scale=True}.  
* The ```scattercut``` argument takes a probe wavelength interval that is ignored (set to zero) in the plots, to suppress the plotting of scattered excitation light. Here the scatter region was found to be between 380 and 405 nm (excitation at 400 nm).
* The ```time_width_percent``` variable is set to 5%, meaning that the transient spectra are shown at the given delay time plus/minus 5% of that value (e.g. 0.522 ps means 0.5 to 0.55 ps). The respective range is indicated in the legend of the transient spectra. In all plots the unfitted data is plotted as dots, interpolated with lines (Savitzky-Golay).

In [ ]:
ta.rel_time=[0.5,1.5,20,100,1000]  # certain delay times for TA spectra plot
ta.rel_wave=[350,440,520,600]      # certain probe wavelengths for kinetic traces plot 
ta.timelimits=[-1,1400]            # plotted delay time range
ta.bordercut=[320,770]             # plotted probe wavelength range
ta.intensity_range=[-55e-3,55e-3]  # plotted intensity range
ta.scattercut=[378,407]            # ignored probe wavelength region (set to zero)
ta.time_width_percent=5            # number in percent defining a delay time region 
                                   # plotted in the TA spectra

ta.Plot_RAW(title='Kinetic traces at selected probe wavelengths', plotting=1)
ta.Plot_RAW(title='TA spectra at selected delay-times', plotting=2)
ta.Plot_RAW(title='2D-Plot', plotting=0)

## 4) Fitting of the data 

### 4a) Global analysis $-$ parallel Model

Upon photoexcitation of [(tbbpy)$_2$Ru(dppz)]$^{2+}$ (**Ru-dppz**) at 400 nm an ensemble of metal-to-ligand charge-transfer (MLCT) states localized in both ligand spheres, *i.e.*, $^1$MLCT$_{tbbpy}$ and $^1$MLCT$_{dppz}$ is populated. Extensive photophysical studies revealed that the subsequent excited state dynamics is determined by the polarity and hydrogen bond donor ability of the surrounding solvent molecules. It was found that long-lived emissive states are populated in polar aprotic solvents. However, this emission switches *off* when the molecules interact with water. This interesting property is based on a solvent sensitive excited state equilibrium between a non-emissive and an emissive state localized on the phenazine (phz) and phenanthroline (phen) moiety of the dppz ligand. 

Extensive photophysical studies in acetonitrile revealed that due to the stabiization of the charge-transfer excited states, the *dark* phz-centered state is formed from the *bright* $^3$MLCT$_{phen}$ state and decays back to the ground state on the sub-ns timescale. This formation of a long-lived long-lived $^3$MLCT$_{phen}$ state is manifested, *i.e.*, in the spectral changes at 340 and 580 nm, which can be quantitatively described by two characteristic time-constants: the first one associated with  intersystem crossing, vibrational cooling and interligand hopping and a second one attributed to the non-radiative decay of a subset of $^3$MLCT states with excess electron density on the phenazine sphere of the dppz ligand ($^3$MLCT$_{phz}$), ultimately populating the long-lived $^3$MLCT$_{phen}$ state. Hence, the three kinetic parameters ```k0```, ```k1``` and ```k2``` are added to the the parameter object. The value for ```k2``` is fixed to 180 ns as obtained from nanosecond time-resolved studies.

![Schematic sketch of the parallel model](img/Fig1_parallel_model.png "Parallel Model")

In [ ]:
# Define fit parameters
ta.par=lmfit.Parameters()
# rate constants
ta.par.add('k0',value=1/2, min=1/10.0, max=1/0.25, vary=True)    
ta.par.add('k1',value=1/150, min=1/200.0, max=1/10.0, vary=True)     
ta.par.add('k2',value=1/100000, vary=False)                         

# time-zero parameter fixed during fit
ta.par.add('t0',value=0.0, min=-0.1, max=0.1, vary=False)   

# pump-pulse width parameter fixed during fit
ta.par.add('resolution', value=0.12, min=0.04, max=0.20, vary=False) 

# Select a in-build model (here: independent parallel decay)
ta.mod='exponential'        

# set delay-time range for fit
ta.timelimits=[0.35,2000]   

# global lifetime analysis (kinetic modeling)
ta.Fit_Global()

### 4a) Repeat the global analysis to estimate the errors 

In [ ]:
# fit-error estimation in a confidence interval of 95%
ta.Fit_Global(confidence_level=0.95) 

### Plotting of error Analysis (advanced handling)
Plotting of the results of the error analysis is challenging and due to the potential large amount of combinations not possible to perform automatic. However here is an example on a single parameter

In [ ]:
ta_listen=[ta.Copy(),ta.Copy()] #create a list for comparision
#the Filename can be manipulated to use the automatic naming 
ta_listen[0].filename="upper confidence limit"
ta_listen[1].filename="lower confidence limit"

for i in range(2):
    #short name for the calculated results for reduced writing
    par=ta.re['fit_results_rates'].copy() 
    if i == 0:
        #overwrite the value with the limits
        par.loc['k0','value']=par.loc['k0','upper_limit']
    else:
        par.loc['k0','value']=par.loc['k0','lower_limit']
    # Write the fit results as input parameter
    ta_listen[i].par=pf.pardf_to_par(par) 
    for key in ta_listen[i].par.keys():
        # Lock the parameter so that only the spectra are calculated
        ta_listen[i].par[key].vary=False  
    # Run the global fit to calculate the new spectra
    ta_listen[i].Fit_Global()   

### Shaping of the plot

The plot could be re-shaped using the plotting GUI. Here we need to use a trick to achieve this automatically. Firstly, we get a handle to the drawn axis, so that we can directly manipulate the plot. Secondly, we set the ylimit of the plot.

In [ ]:
ta.Compare_at_wave(fitted=True, 
                   other=ta_listen, 
                   rel_wave=[450,590], 
                   width=50, 
                   linewidth=3)
 
ax=plt.gca()              # get a handle to the drawn axis   
ax.set_ylim(-50e-3,20e-3)  # set the ylimit of the plot

### 4b) Target analysis $-$ propose a model

Several studies in acetonitrile reveal that the *dark* phz-centered excited state is populated from the initially excited $^3$MLCT states. Ultimately, a *bright* $^3$MLCT$_{phen}$ is formed from the tbbpy, phen and phz centered states. Herein, it is shown how to define an own model function based on such *a priori* knowledge.

### 4b.1) Define own model function

Based on the literature findings a model is proposed, where initially proximal MLCT (tbbpy and phen, A) and distal MLCT states (phz, B) are populated. Subsequently those states decay forming the *bright* MLCT$_{phen}$ state (C). This in turn decays back to the ground state (C'). The respective kinetic rate constants can be written as

- $\displaystyle \frac{d[A]}{dt} = -k_0 \cdot [A]$
- $\displaystyle \frac{d[B]}{dt} = -k_0 \cdot [B]$
- $\displaystyle \frac{d[C]}{dt} = k_0 \cdot ([A] + [B]) -k_1 \cdot [C]$,

with brackets indicating the concentration of A, B and C. Those rate equations are defined in the python function, like:

```python
# state: A, d[A]/dt
dc[0] = -pardf['k0']*dt*c_temp[0] + g[i]*dt
# state: B, d[B]/dt
dc[1] = -pardf['k0']*dt*c_temp[1] + g[i]*dt
# state: C, d[C]/dt
dc[2] = pardf['k0']*dt*c_temp[0] + pardf['k0']*dt*c_temp[1] - pardf['k1']*dt*c_temp[2]
                        
```

![Schematic sketch of the user-defined model](img/Fig3_complex_model.png "Parallel Model")

In [ ]:
FWHM=2.35482

def gauss(t,sigma=0.1,mu=0):
    y=np.exp(-0.5*((t-mu)**2)/sigma**2)
    y/=sigma*np.sqrt(2*np.pi)
    return y

def Rudppz(times,pardf):
        '''
        Define a model where initially A and B are populated and decay forming C. 
        Subsequently, C decays back to the ground-state (C').
        
        args:
                pardf: pandas.DataFrame with the column 'value' named in 'pardf' (type: 
                pandas.DataFrame)
                times: vector (type:list)
                
        returns:
                c: DataFrame with the times as index and in the columns the an expression
                of the relative concentrations of A, B and C (type: dictionary)
        '''
        # create an empty concentration matrix
        c=np.zeros((len(times),3),dtype='float')
        # create IRF                                            
        g=gauss(times,sigma=pardf['resolution']/FWHM,mu=pardf['t0'])
        # defining how many extra steps are taken between the main time_points
        sub_steps=10
        for i in range(1,len(times)):
                # initial change for each concentration (3 refers to the number of states)                               
                dc=np.zeros((3,1),dtype='float')
                # adaption of the time-intervals to the sub_steps
                dt=(times[i]-times[i-1])/(sub_steps)
                # create a temporary concentration matrix
                c_temp=c[i-1,:]
                for j in range(int(sub_steps)):
                        # state: A, d[A]/dt
                        dc[0] = -pardf['k0']*dt*c_temp[0] + g[i]*dt
                        # state: B, d[B]/dt
                        dc[1] = -pardf['k0']*dt*c_temp[1] + g[i]*dt
                        # state: C, d[C]/dt
                        dc[2] = pardf['k0']*dt*c_temp[0] + pardf['k0']*dt*c_temp[1] - pardf['k1']*dt*c_temp[2]
                        for b in range(c.shape[1]):
                                #check that all concentrations are > 0
                                c_temp[b] =np.nanmax([(c_temp[b]+dc[b]),0.])
                # store temporary concentrations into the main matrix
                c[i,:] =c_temp
        c=pd.DataFrame(c,index=times)
        c.index.name='time' # name the delay-times
        c.columns=['A','B','C'] # name the species

        return c


### 4b.2) Define fitting parameters

In [ ]:
ta.par=lmfit.Parameters()
# rate constants
ta.par.add('k0',value=1/2.0, min=1/10.0, max=1/0.1, vary=True)
ta.par.add('k1',value=1/100000, vary=False)

# time-zero parameter fixed during fit
ta.par.add('t0',value=0.0, min=-0.1, max=0.1, vary=False) 

# pump-pulse width parameter fixed during fit
ta.par.add('resolution', value=0.07, min=0.04, max=0.20, vary=False) 

### 4b.3) Fitting of the Data $-$ Kinetic Modeling (**Ru-dppz** Model)

In [ ]:
ta.mod=Rudppz               # model selection (own model)

ta.timelimits=[0.35,2000]   # set delay-time range for fit
ta.log_fit=False            # fitting on linear time scale

ta.Fit_Global()             # pass parameter object (par) to global fit 

## 5) Plot the fit results

In [ ]:
plt.close('all')
ta.Plot_fit_output(title='2D-Plots', plotting=4)

In [ ]:
ta.Plot_fit_output(title='summed TA signals', plotting=1)
ta.Plot_fit_output(title='Decay Associated Spectra', plotting=0)
ta.Plot_fit_output(title='concentration profiles', plotting=5) #or: ta.re['c'].plot()

## 6) Save results

In [ ]:
savename = filename+'_own'
ta.Save_project(filename=savename,   # set save name
                path='results')   # set name of save folder

ta.Save_data(save_RAW=False,         # do not save pre-processed data
             save_Fit=True,          # save pre-processed and fitted data
             filename=savename,      # set save name
             path='results')      # set name of save folder